In [1]:
import json

from kafka import KafkaConsumer

### Configuration Parameters 

> **TODO:** Change the configuration prameters to the appropriate values for your setup.

In [2]:
config = dict(
    bootstrap_servers=['kafka.kafka.svc.cluster.local:9092'],
    first_name='Josh',
    last_name='Greenert'
)

config['client_id'] = '{}{}'.format(
    config['last_name'], 
    config['first_name']
)
config['topic_prefix'] = '{}{}'.format(
    config['last_name'], 
    config['first_name']
)

config

{'bootstrap_servers': ['kafka.kafka.svc.cluster.local:9092'],
 'first_name': 'Josh',
 'last_name': 'Greenert',
 'client_id': 'GreenertJosh',
 'topic_prefix': 'GreenertJosh'}

Create a consumer without subscribing to any particular topic

In [3]:
general_consumer = KafkaConsumer(
    bootstrap_servers=config['bootstrap_servers']
)

List all topics you are currently allowed to view

In [4]:
general_consumer.topics()

{'AvinazGabe-simple',
 'AvinazGabriel-accelerations',
 'AvinazGabriel-joined',
 'AvinazGabriel-locations',
 'AvinazGabriel-simple',
 'AvinazGabriel-windowed',
 'BondaAnjani-accelerations',
 'BondaAnjani-locations',
 'ChristudassReenie-accelerations',
 'ChristudassReenie-locations',
 'ChristudassReenie-simple',
 'ClaytonDan-joined',
 'ClaytonDan-simple',
 'ClaytonDan-windowed',
 'ClaytonDaniel-accelerations',
 'ClaytonDaniel-locations',
 'ClaytonDaniel-simple',
 'CosentiniVictor-accelerations',
 'CosentiniVictor-joined',
 'CosentiniVictor-locations',
 'CosentiniVictor-simple',
 'CosentiniVictor-windowed',
 'DoeJohn-accelerations',
 'DoeJohn-joined',
 'DoeJohn-locations',
 'DoeJohn-simple',
 'DoeJohn-windowed',
 'GreenertJosh-accelerations',
 'GreenertJosh-joined',
 'GreenertJosh-locations',
 'GreenertJosh-simple',
 'GreenertJosh-windowed',
 'HanPhil-accelerations',
 'HanPhil-locations',
 'HanPhil-simple',
 'HanPhil-windowed',
 'HanSangPhil-accelerations',
 'HanSangPhil-locations',
 'Her

Close the consumer, waiting indefinitely for any needed cleanup.

In [5]:
general_consumer.close()

In [6]:
def create_kafka_consumer(topics, config=config):
    bootstrap_servers = config['bootstrap_servers']
    client_id = config['client_id']
    topic_prefix = config['topic_prefix']    
    topic_list = ['{}-{}'.format(topic_prefix, topic) for topic in topics]
    
    return KafkaConsumer(
        *topic_list,
        client_id=client_id,
        bootstrap_servers=bootstrap_servers,
        auto_offset_reset='earliest',
        enable_auto_commit=False,
        value_deserializer=lambda x: json.loads(x)
    )

consumer = create_kafka_consumer(['locations', 'accelerations'])

Gets a list of this consumer's current subscriptions

In [7]:
consumer.subscription()

{'GreenertJosh-accelerations', 'GreenertJosh-locations'}

The following function prints messages from the current consumer subscriptions. It will continue until manually stopped. 

In [ ]:
def print_messages(consumer=consumer):
    try:
        for message in consumer:
                msg_metadata = 'Message metadata: {}:{}:{}'.format(
                    message.topic, message.partition, message.offset
                )

                if message.key is not None:
                    msg_key = message.key.decode('utf-8')
                else:
                    msg_key = ''
                msg_value = json.dumps(message.value, indent=2)
                msg_value = '\n'.join(['  {}'.format(value) for value in msg_value.split('\n')])

                print('Message metadata:')
                print('  Topic: {}'.format(message.topic))
                print('  Partition: {}'.format(message.partition))
                print('  Offset: {}'.format(message.offset))
                print('Message Key: {}'.format(msg_key))
                print('Message Value:')
                print(msg_value)
                print()
    except KeyboardInterrupt:
        print("STOPPING MESSAGE CONSUMER")
        
print_messages()

Close the consumer, waiting indefinitely for any needed cleanup.

In [ ]:
# Close the consumer since it's done processing the messages.
consumer.close()